# Using hypothesis to find interesting examples

Hypothesis is a powerful and unique library for testing code. It also includes a `find` function for finding examples that satisfy an arbitrary predicate. Here, we will explore some of the neat things that can be found using this function.

In [3]:
from hypothesis import find

import dit
from dit.abc import *
from dit.pid import *
from dit.utils.testing import distribution_structures

dit.ditParams['repr.print'] = dit.ditParams['print.exact'] = True

To illustrate what the distribution source looks like, here we instantiate it with a `size` of 3 and an alphabet of `2`:

In [4]:
a = distribution_structures(size=3, alphabet=2)

In [5]:
a.example()

Class:          Distribution
Alphabet:       (0, 1) for all rvs
Base:           linear
Outcome Class:  tuple
Outcome Length: 3
RV Names:       None

x           p(x)
(0, 0, 0)   2667/18926
(0, 0, 1)   4354/23287
(0, 1, 0)   2735/34879
(0, 1, 1)   5185/28822
(1, 0, 0)   1049/14724
(1, 0, 1)   5043/26701
(1, 1, 0)   13331/95672
(1, 1, 1)   775/54022

## Negativity of co-information

In [11]:
def pred(value):
    return lambda d: dit.multivariate.coinformation(d) < value

In [12]:
ce = find(distribution_structures(3, 2), pred(-1e-5))
print(ce)
print(f"The coinformation is: {dit.multivariate.coinformation(ce)}")

Class:          Distribution
Alphabet:       (0, 1) for all rvs
Base:           linear
Outcome Class:  tuple
Outcome Length: 3
RV Names:       None

x           p(x)
(0, 0, 0)   1/3
(0, 1, 1)   1/3
(1, 0, 1)   1/3
The coinformation is: -0.4150374992788435


In [13]:
ce = find(distribution_structures(3, 2), pred(-0.5))
print(ce)
print(f"The coinformation is: {dit.multivariate.coinformation(ce)}")

Class:          Distribution
Alphabet:       (0, 1) for all rvs
Base:           linear
Outcome Class:  tuple
Outcome Length: 3
RV Names:       None

x           p(x)
(0, 0, 0)   1/4
(0, 1, 1)   1/4
(1, 0, 1)   1/4
(1, 1, 0)   1/4
The coinformation is: -1.0


## The Gács-Körner common information is bound from above by the dual total correlation

As we will see, hypothesis can not find an example of $K > B$, because one does not exist.

In [14]:
def b_lt_k(d):
    k = dit.multivariate.gk_common_information(d)
    b = dit.multivariate.dual_total_correlation(d)
    return  k > b

In [15]:
find(distribution_structures(size=3, alphabet=3, uniform=True), b_lt_k)

NoSuchExample: No examples found of condition b_lt_k

## BROJA is not Proj

We know that the BROJA and Proj PID measures are not the same, but the BROJA paper did not provide any simple examples of this. Here, we find one.

In [16]:
ce = find(distribution_structures(3, 2, True), lambda d: PID_BROJA(d) != PID_Proj(d))
ce

Class:          Distribution
Alphabet:       (0, 1) for all rvs
Base:           linear
Outcome Class:  tuple
Outcome Length: 3
RV Names:       None

x           p(x)
(0, 1, 0)   1/4
(1, 0, 1)   1/4
(1, 1, 0)   1/4
(1, 1, 1)   1/4

In [17]:
print(PID_BROJA(ce))
print(PID_Proj(ce))

+---------+--------+--------+
| I_broja |  I_r   |   pi   |
+---------+--------+--------+
|  {0:1}  | 0.5000 | 0.0000 |
|   {0}   | 0.3113 | 0.1887 |
|   {1}   | 0.3113 | 0.1887 |
|  {0}{1} | 0.1226 | 0.1226 |
+---------+--------+--------+
+--------+--------+--------+
| I_proj |  I_r   |   pi   |
+--------+--------+--------+
| {0:1}  | 0.5000 | 0.0425 |
|  {0}   | 0.3113 | 0.1462 |
|  {1}   | 0.3113 | 0.1462 |
| {0}{1} | 0.1650 | 0.1650 |
+--------+--------+--------+
